<a href="https://colab.research.google.com/github/lmcanavals/acomplex/blob/main/1303_johnson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math

def bellmanFord(G, s): # returns the path π and and whether or not a negative cycle exists
    n = len(G)                      # |V|
    π = [-1]*n                      # path
    g = [math.inf]*n                # cost
    g[s] = 0

    for _ in range(n-1):            # repetir |V| - 1 veces
        for u in range(n):
            for v, w in G[u]:       # para cada arco (u, v, con peso w) de E
                f = g[u] + w
                if f < g[v]:        # relax(u, v, w)
                    g[v] = f
                    π[v] = u

    
    for u in range(n):              # para cada arco (u, v, con peso w) de E
        for v, w in G[u]:
            f = g[u] + w
            if f < g[v]:            # se puede relajar?
                return None         # existe ciclo negativo

    return g                        # no existe ciclo negativo

In [3]:
import heapq as hq

def dijkstra(G, s, path, cost):
  n = len(G)
  visited = [False]*n

  cost[s] = 0
  pqueue = [(0, s)]
  while pqueue:
    g, u = hq.heappop(pqueue)
    if not visited[u]:
      visited[u] = True
      for v, w in G[u]:
        if not visited[v]:
          f = g + w
          if f < cost[v]:
            cost[v] = f
            path[v] = u
            hq.heappush(pqueue, (f, v))

  return path, cost

In [5]:
import numpy as np

def johnson(G):
    n = len(G)
    G.append([(0, 0)])
    g = bellmanFord(G, n)
    if g == None: return None
    del G[n]
    for u in range(n):
        for i in range(len(G[u])):
            v, w = G[u][i]
            w = w + g[u] - g[v]
            G[u][i] = (v, w)

    path = np.full((n, n), -1, dtype=int)
    cost = np.full((n, n), math.inf)
    for u in range(n):
        dijkstra(G, u, path[u], cost[u])

    return path, cost


In [6]:
%%file g.g
1 2
2 -1
0 4 3 2 4 -4
-
-
3 1 4 -4

Writing g.g


In [7]:
with open("g.g") as f:
    G = []
    for line in f:
        if line.startswith('-'):
            G.append([])
        else:
            nums = [int(x) for x in line.split()]
            G.append([(nums[i], nums[i+1]) for i in range(0, len(nums), 2)])

for x in G:
  print(x)

[(1, 2)]
[(2, -1)]
[(0, 4), (3, 2), (4, -4)]
[]
[]
[(3, 1), (4, -4)]


In [8]:
path, cost = johnson(G)
print(path)
print(cost)

[[-1  0  1  2  2 -1]
 [ 2 -1  1  2  2 -1]
 [ 2  0 -1  2  2 -1]
 [-1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1 -1 -1]]
[[ 0.  0.  0.  0.  0. inf]
 [ 5.  0.  0.  0.  0. inf]
 [ 5.  5.  0.  0.  0. inf]
 [inf inf inf  0. inf inf]
 [inf inf inf inf  0. inf]
 [inf inf inf inf inf  0.]]
